# Código para o cálculo do tensor de Einstein

A partir de uma métrica podemos calcular os símbolos de Christoffel.

$\Gamma^{\mu}_{\nu\lambda}=\frac{1}{2}g^{\mu\alpha}(\partial_\lambda g_{\nu\alpha}+\partial_\nu g_{\lambda\alpha}-\partial_\alpha g_{\nu\lambda})$

Com os símbolos podemos calcular os tensores de Ricci. 

$R_{\mu\nu}=\partial_\alpha \Gamma^{\alpha}_{\mu\nu}+\Gamma^{\lambda}_{\mu\nu}\Gamma^{\alpha}_{\lambda\alpha}-\partial_\nu\Gamma^{\alpha}_{\nu\alpha}-\Gamma^{\alpha}_{\lambda\nu}\Gamma^{\lambda}_{\mu\alpha}$

Com os tensores de Ricci e a métrica, podemos calcular o escalar de Ricci.

$R=g^{\mu\nu}R_{\mu\nu}$

Com os tensores de Ricci, a métrica e o escalar de Ricci, calculamos o tensor de Einstein.

$G_{\mu\nu}=R_{\mu\nu}-\frac{1}{2}g_{\mu\nu}R$

## Métrica

In [4]:
import sympy as sp
sp.init_printing()

# Definição dos símbolos e variáveis
t, r, theta, phi = sp.symbols('t r theta phi')
A, B = sp.symbols('A B', cls=sp.Function)

# Definição da métrica
metric = sp.Matrix([
    [-A(r), 0, 0, 0],
    [0, B(r), 0, 0],
    [0, 0, r**2, 0],
    [0, 0, 0, r**2*sp.sin(theta)**2]
])

# Mostrando a métrica
sp.pprint(metric)

# Inverso da métrica e mostrando o inverso da métrica
metric_inv = metric.inv()
sp.pprint(metric_inv)

⎡-A(r)   0    0       0     ⎤
⎢                           ⎥
⎢  0    B(r)  0       0     ⎥
⎢                           ⎥
⎢              2            ⎥
⎢  0     0    r       0     ⎥
⎢                           ⎥
⎢                  2    2   ⎥
⎣  0     0    0   r ⋅sin (θ)⎦
⎡-1                        ⎤
⎢────   0    0       0     ⎥
⎢A(r)                      ⎥
⎢                          ⎥
⎢       1                  ⎥
⎢ 0    ────  0       0     ⎥
⎢      B(r)                ⎥
⎢                          ⎥
⎢            1             ⎥
⎢ 0     0    ──      0     ⎥
⎢             2            ⎥
⎢            r             ⎥
⎢                          ⎥
⎢                    1     ⎥
⎢ 0     0    0   ──────────⎥
⎢                 2    2   ⎥
⎣                r ⋅sin (θ)⎦


## Símbolos de Christoffel

In [2]:
# Derivadas da métrica em relação às componentes
dmetric = [metric.diff(x) for x in (t, r, theta, phi)]

# Definindo a função que chamará o símbolo de Christoffel
def christoffel(i,j,k):
    return sum(0.5 * metric_inv[i, a] * (dmetric[k][j, a] + dmetric[j][k,a] - dmetric[a][j, k])  for a in range(4))

# Printando os não nulos
for i in range(4):
    for j in range(4):
        for k in range(4):
            if christoffel(i,j,k)!=0:
                print(f"Christoffel({i},{j},{k}): {sp.simplify(christoffel(i,j,k))}")
            else:
                pass

Christoffel(0,0,1): 0.5*Derivative(A(r), r)/A(r)
Christoffel(0,1,0): 0.5*Derivative(A(r), r)/A(r)
Christoffel(1,0,0): 0.5*Derivative(A(r), r)/B(r)
Christoffel(1,1,1): 0.5*Derivative(B(r), r)/B(r)
Christoffel(1,2,2): -1.0*r/B(r)
Christoffel(1,3,3): -1.0*r*sin(theta)**2/B(r)
Christoffel(2,1,2): 1.0/r
Christoffel(2,2,1): 1.0/r
Christoffel(2,3,3): -0.5*sin(2*theta)
Christoffel(3,1,3): 1.0/r
Christoffel(3,2,3): 1.0/tan(theta)
Christoffel(3,3,1): 1.0/r
Christoffel(3,3,2): 1.0/tan(theta)


## Tensores de Ricci

In [3]:
# Definindo a derivada dos símbolos
def dChristoffel(d,i,j,k):
    if d==0:
        return christoffel(i,j,k).diff(t)
    elif d==1:
        return christoffel(i,j,k).diff(r)
    elif d==2:
        return christoffel(i,j,k).diff(theta)
    elif d==3:
        return christoffel(i,j,k).diff(phi)

# Definindo o tensor de Ricci com apenas um índice mudo
def ricci_a(a,i,j):
    if a==0:
        return dChristoffel(0,0,i,j) + sum(christoffel(0,l,0)*christoffel(l,i,j)for l in range(4))\
             - dChristoffel(j,0,i,0) - sum(christoffel(0,l,j)*christoffel(l,i,0)for l in range(4))
    elif a==1:
        return dChristoffel(1,1,i,j) + sum(christoffel(1,l,1)*christoffel(l,i,j)for l in range(4))\
             - dChristoffel(j,1,i,1) - sum(christoffel(1,l,j)*christoffel(l,i,1)for l in range(4))
    elif a==2:
        return dChristoffel(2,2,i,j) + sum(christoffel(2,l,2)*christoffel(l,i,j)for l in range(4))\
             - dChristoffel(j,2,i,2) - sum(christoffel(2,l,j)*christoffel(l,i,2)for l in range(4))
    elif a==3:
        return dChristoffel(3,3,i,j) + sum(christoffel(3,l,3)*christoffel(l,i,j)for l in range(4))\
             - dChristoffel(j,3,i,3) - sum(christoffel(3,l,j)*christoffel(l,i,3)for l in range(4))
    
# Definindo o tensor de Ricci abreviado
def ricci(i,j):
    return sum(ricci_a(a,i,j) for a in range(4))

# Printando os tensores de Ricci não nulos
for i in range(4):
    for j in range(4):
        if ricci(i,j)!=0:
            print(f"Ricci({i},{j}):{sp.simplify(ricci(i,j))}")
        else:
            pass

Ricci(0,0):0.5*Derivative(A(r), (r, 2))/B(r) - 0.25*Derivative(A(r), r)*Derivative(B(r), r)/B(r)**2 - 0.25*Derivative(A(r), r)**2/(A(r)*B(r)) + 1.0*Derivative(A(r), r)/(r*B(r))
Ricci(1,1):-0.5*Derivative(A(r), (r, 2))/A(r) + 0.25*Derivative(A(r), r)*Derivative(B(r), r)/(A(r)*B(r)) + 0.25*Derivative(A(r), r)**2/A(r)**2 + 1.0*Derivative(B(r), r)/(r*B(r))
Ricci(2,2):0.5*r*Derivative(B(r), r)/B(r)**2 - 0.5*r*Derivative(A(r), r)/(A(r)*B(r)) + 1.0 - 1.0/B(r)
Ricci(3,3):(0.5*r*A(r)*Derivative(B(r), r) - 0.5*r*B(r)*Derivative(A(r), r) + 1.0*A(r)*B(r)**2 - 1.0*A(r)*B(r))*sin(theta)**2/(A(r)*B(r)**2)


## Escalar de Ricci

In [4]:
# Escalar para um índice mudo
escalar_ricci_0=sum(metric_inv[0,j]*ricci(0,j) for j in range(4))
escalar_ricci_1=sum(metric_inv[1,j]*ricci(1,j) for j in range(4))
escalar_ricci_2=sum(metric_inv[2,j]*ricci(2,j) for j in range(4))
escalar_ricci_3=sum(metric_inv[3,j]*ricci(3,j) for j in range(4))

# Escalar total
escalar_ricci=escalar_ricci_0+escalar_ricci_1+escalar_ricci_2+escalar_ricci_3

# Mostrando
print(f"Escalar de Ricci = {sp.simplify(escalar_ricci)}")

Escalar de Ricci = -1.0*Derivative(A(r), (r, 2))/(A(r)*B(r)) + 0.5*Derivative(A(r), r)*Derivative(B(r), r)/(A(r)*B(r)**2) + 0.5*Derivative(A(r), r)**2/(A(r)**2*B(r)) + 2.0*Derivative(B(r), r)/(r*B(r)**2) - 2.0*Derivative(A(r), r)/(r*A(r)*B(r)) + 2.0/r**2 - 2.0/(r**2*B(r))


## Tensores de Einstein

In [5]:
# Definindo o tensor
def einstein(i,j):
    return ricci(i,j)-0.5*metric[i,j]*escalar_ricci

# Printandos os tensores de Einstein não nulos
for i in range(4):
    for j in range(4):
        if einstein(i,j)!=0:
            print(f"Einsteins({i},{j}): {sp.simplify(einstein(i,j))}")
        else:
            pass

Einsteins(0,0): 1.0*(r*Derivative(B(r), r) + B(r)**2 - B(r))*A(r)/(r**2*B(r)**2)
Einsteins(1,1): 1.0*Derivative(A(r), r)/(r*A(r)) - 1.0*B(r)/r**2 + 1.0/r**2
Einsteins(2,2): 1.0*r*(0.5*r*A(r)*B(r)*Derivative(A(r), (r, 2)) - 0.25*r*A(r)*Derivative(A(r), r)*Derivative(B(r), r) - 0.25*r*B(r)*Derivative(A(r), r)**2 - 0.5*A(r)**2*Derivative(B(r), r) + 0.5*A(r)*B(r)*Derivative(A(r), r))/(A(r)**2*B(r)**2)
Einsteins(3,3): 1.0*r*(0.5*r*A(r)*B(r)*Derivative(A(r), (r, 2)) - 0.25*r*A(r)*Derivative(A(r), r)*Derivative(B(r), r) - 0.25*r*B(r)*Derivative(A(r), r)**2 - 0.5*A(r)**2*Derivative(B(r), r) + 0.5*A(r)*B(r)*Derivative(A(r), r))*sin(theta)**2/(A(r)**2*B(r)**2)
